In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [5]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [6]:
async def get_expiry(f_today):

    if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
        f_expiry = dt.date(2024, 1, 25)
    elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
        f_expiry = dt.date(2024, 1, 31)
    elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
        f_expiry = dt.date(2024, 2, 29)
    elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
        f_expiry = dt.date(2024, 2, 27)
    elif f_today < dt.date(2023, 9, 1):
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
    elif f_today >= dt.date(2023, 9, 1):
        if f_today.day < 24:
            days_to_wednesday = (2 - f_today.weekday()) % 7
            nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
            f_expiry = nearest_wednesday
            if nse.valid_days(
                start_date=nearest_wednesday, end_date=nearest_wednesday
            ).empty:
                f_expiry = nearest_wednesday - dt.timedelta(days=1)
        else:
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(
                start_date=nearest_thursday, end_date=nearest_thursday
            ).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_finnifty(f_today):

    days_to_thursday = (1 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [7]:
# bnf_pandas = pd.read_csv('../data/nifty_1hr_tv (2).csv')
# bnf_pandas = pd.read_csv('../data/bnf_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/midcp_select_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/sensex_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/crude_4hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/gold_4hr_tv.csv')
# bnf_1min = pd.read_csv('../data/bnf_min.csv')
bnf_1min = pd.read_csv("../data/finnifty_min.csv")
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year == 2020) & (bnf_1min['datetime'].dt.month == 4))]
bnf_1min = bnf_1min[
    (bnf_1min["datetime"].dt.year >= 2022) & (bnf_1min["datetime"].dt.year <= 2024)
]

In [8]:
# # If Stocks Data ...
# bnf_pandas['datetime'] = pd.to_datetime(bnf_pandas['time'])
# bnf_pandas['datetime'] = bnf_pandas['datetime'].dt.tz_localize(None)
# bnf_pandas = bnf_pandas[bnf_pandas['datetime'].dt.year >= 2017]
# bnf_pandas.drop(columns=['time'], inplace=True)
# # bnf_pandas

In [9]:
# bnf = pl.DataFrame(bnf_pandas)
# bnf = bnf.with_columns([pl.col('datetime').alias('index')]).drop('datetime')
# bnf = bnf.with_columns(pl.col("index").alias("datetime"))
# # bnf

In [10]:
# bnf_1min

In [11]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [33]:
def calculate_weekly_ranges(df):
    # Ensure the index is a DatetimeIndex
    df.index = pd.to_datetime(df.index)
    # Resample to weekly data to get the high and low for each week
    weekly_ranges = df.resample("W-FRI").agg({"high": "max", "low": "min"})
    return weekly_ranges


async def find_breakouts(df, weekly_ranges):

    results = []
    multiplier_to_range = 0.4

    portfolio_value = 10_00_000
    index_leverage = 8

    index_name = "FINNIFTY"
    index_str_for_opt = "finnifty"

    weekly_ranges["datetime"] = pd.to_datetime(weekly_ranges["datetime"])
    weekly_ranges = weekly_ranges[:-2]
    # print(weekly_ranges)

    dates_to_avoid = [
        # dt.date(2017, 1, 23),
        # dt.date(2017, 3, 13),
        # dt.date(2017, 4, 10),
        # dt.date(2017, 12, 11),
        # dt.date(2019, 3, 25),
        # dt.date(2019, 4, 1),
        # dt.date(2019, 9, 9),
        # dt.date(2020, 4, 6),
        # dt.date(2020, 10, 26),
    ]

    for i in range(1, len(weekly_ranges)):
        previous_week = weekly_ranges.iloc[i - 1]
        current_week = weekly_ranges.iloc[i]
        print("Current Week : ", current_week["datetime"].date())
        if current_week["datetime"].date() in dates_to_avoid:
            print("Date Avoided")
            continue
        previous_week_high = weekly_ranges.iloc[i - 1]["high"]
        previous_week_low = weekly_ranges.iloc[i - 1]["low"]
        previous_week_close = weekly_ranges.iloc[i - 1]["close"]

        current_week_open = weekly_ranges.iloc[i]["open"]

        # Filter the 1-minute data for the current week
        # current_week_data = df[(df.index > weekly_ranges.index[i-1]) & (df.index <= current_week)]
        current_week_data = df.loc[
            (df["datetime"] >= current_week["datetime"])
            & (df["datetime"] <= (current_week["datetime"] + pd.Timedelta(days=7)))
        ]
        # print("Current Week Data:\n", current_week_data)

        weekly_range = previous_week_high - previous_week_low
        addition_range = multiplier_to_range * weekly_range

        # Check for breakouts
        high_level = current_week_data["open"].iloc[0] + addition_range
        low_level = current_week_data["open"].iloc[0] - addition_range

        # high_level = current_week_data['open'].iloc[0] + addition_range
        # low_level = current_week_data['open'].iloc[0] - addition_range
        # print(current_week)
        # print("High Level:\n", high_level, "\nLow Level:\n", low_level)

        breakout_high = current_week_data[current_week_data["high"] >= high_level]
        breakout_low = current_week_data[current_week_data["low"] <= low_level]

        if not breakout_high.empty:
            # print('Breakout High')
            breakout_high_time = breakout_high.iloc[0]["datetime"]
            # print('High Break At :', breakout_high_time)
            atm_strike = int(round(low_level / 50) * 50)
            # exit_date = current_week['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry_finnifty(
                current_week["datetime"].date() + dt.timedelta(days=5)
            )
            dte = (expiry - breakout_high_time.date()).days
            contract = await get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type="PE",
            )
            pe_df = await fetch_data(
                index=index_str_for_opt,
                start_date=breakout_high_time.date(),
                end_date=expiry,
                start_time=breakout_high_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=atm_strike,
                asset_class="P",
            )
            if not isinstance(pe_df, str) and pe_df is not None:
                pe_df = pe_df.to_pandas()
                entry_price = pe_df.iloc[0]["c"]
            else:
                entry_price = float("nan")

            # Exit Scenario
            current_week_data_after_entry = current_week_data[
                (current_week_data["datetime"] > breakout_high_time)
                & (
                    current_week_data["datetime"].dt.date
                    <= expiry - dt.timedelta(days=0)
                )
            ]
            # print(current_week_data_after_entry)
            low_breach = current_week_data_after_entry[
                current_week_data_after_entry["low"] < previous_week_low
            ]
            # print(low_breach)
            if len(low_breach) != 0:
                exit_time = low_breach.iloc[0]["datetime"]
                # print(exit_time)
                remark = "SL Hit"
                # print(remark, exit_time)
                # print(pe_df)
                if not math.isnan(entry_price):
                    # print((pe_df[(pe_df['datetime'].dt.date == exit_time.date()) & (pe_df['datetime'].dt.hour >= exit_time.hour) & (pe_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    filtered_df = pe_df.loc[
                        (pe_df["datetime"].dt.date >= exit_time.date())
                        & (pe_df["datetime"].dt.hour >= exit_time.hour)
                        & (pe_df["datetime"].dt.minute >= exit_time.minute),
                        "c"]
                    exit_price = filtered_df.iloc[0] if not filtered_df.empty else float('nan')

                else:
                    exit_price = float("nan")
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(
                    expiry - dt.timedelta(days=0), dt.time(15, 30)
                )
                remark = "Expiry Closing"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = pe_df.iloc[-1]["c"]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)
            qty = portfolio_value * index_leverage / atm_strike
            slippage = 0.01 * (entry_price + exit_price)
            pnl = qty * (entry_price - exit_price)
            final_pnl = qty * (entry_price - exit_price - slippage)
            unit = {
                "Week": previous_week["datetime"].date(),
                "Week High": previous_week_high,
                "Week Low": previous_week_low,
                "Weekly Range": weekly_range,
                "Current Week Open": current_week_open,
                "Multiplier to Range": f"{int(multiplier_to_range * 100)}%",
                "Entry Level Long": high_level,
                "Entry Level Short": low_level,
                "Break Type": "high",
                "Strike": atm_strike,
                "Expiry": expiry,
                "DTE": dte,
                "Contract": contract,
                "Option Type": contract[-2:],
                "Entry Time": breakout_high_time,
                "Entry Price": entry_price,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Remark": remark,
                "Points": entry_price - exit_price,
                "Slippage": slippage,
                "Points w cs": (entry_price - exit_price) - slippage,
                "Qty": qty,
                "PnL": pnl,
                "PnL w cs": final_pnl,
                "ROI%": (pnl * 100 / portfolio_value),
                "ROI% w cs": (final_pnl * 100 / portfolio_value),
            }
            # print(unit)
            results.append(unit)

        if not breakout_low.empty:
            # print('Breakout Low')
            breakout_low_time = breakout_low.iloc[0]["datetime"]
            # print('Low Break At :', breakout_low_time)
            atm_strike = int(round(high_level / 50) * 50)
            # exit_date = current_week['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry_finnifty(
                current_week["datetime"].date() + dt.timedelta(days=5)
            )
            dte = (expiry - breakout_low_time.date()).days
            contract = await get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type="CE",
            )
            ce_df = await fetch_data(
                index=index_str_for_opt,
                start_date=breakout_low_time.date(),
                end_date=expiry,
                start_time=breakout_low_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=atm_strike,
                asset_class="C",
            )
            # print(len(ce_df))
            if not isinstance(ce_df, str) and ce_df is not None:
                ce_df = ce_df.to_pandas()
                entry_price = ce_df.iloc[0]["c"]
            else:
                entry_price = float("nan")

            # Exit Scenario
            current_week_data_after_entry = current_week_data[
                (current_week_data["datetime"] > breakout_low_time)
                & (
                    current_week_data["datetime"].dt.date
                    <= expiry - dt.timedelta(days=0)
                )
            ]
            # print(current_week_data_after_entry)
            high_breach = current_week_data_after_entry[
                current_week_data_after_entry["high"] > previous_week_high
            ]
            # print(high_breach)
            if len(high_breach) != 0:
                exit_time = high_breach.iloc[0]["datetime"]
                # print(exit_time)
                remark = "SL Hit"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    # print(exit_time.date(), exit_time.hour, exit_time.minute)
                    # print((ce_df[(ce_df['datetime'].dt.date == exit_time.date()) & (ce_df['datetime'].dt.hour >= exit_time.hour) & (ce_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    filtered_df = ce_df.loc[
                        (ce_df["datetime"].dt.date >= exit_time.date())
                        & (ce_df["datetime"].dt.hour >= exit_time.hour)
                        & (ce_df["datetime"].dt.minute >= exit_time.minute),
                        "c"]
                    exit_price = filtered_df.iloc[0] if not filtered_df.empty else float('nan') 
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(
                    expiry - dt.timedelta(days=0), dt.time(15, 30)
                )
                remark = "Expiry Closing"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = ce_df.iloc[-1]["c"]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)

            qty = portfolio_value * index_leverage / atm_strike
            slippage = 0.01 * (entry_price + exit_price)
            pnl = qty * (entry_price - exit_price)
            final_pnl = qty * (entry_price - exit_price - slippage)
            unit = {
                "Week": previous_week["datetime"].date(),
                "Week High": previous_week_high,
                "Week Low": previous_week_low,
                "Weekly Range": weekly_range,
                "Current Week Open": current_week_open,
                "Multiplier to Range": f"{int(multiplier_to_range * 100)}%",
                "Entry Level Long": high_level,
                "Entry Level Short": low_level,
                "Break Type": "low",
                "Strike": atm_strike,
                "Expiry": expiry,
                "DTE": dte,
                "Contract": contract,
                "Option Type": contract[-2:],
                "Entry Time": breakout_low_time,
                "Entry Price": entry_price,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Remark": remark,
                "Points": entry_price - exit_price,
                "Slippage": slippage,
                "Points w cs": (entry_price - exit_price) - slippage,
                "Qty": qty,
                "PnL": pnl,
                "PnL w cs": final_pnl,
                "ROI%": (pnl * 100 / portfolio_value),
                "ROI% w cs": (final_pnl * 100 / portfolio_value),
            }
            # print(unit)
            results.append(unit)

    # print(results)
    return results

async def trade():
    df = bnf_1min
    # weekly_ranges = calculate_weekly_ranges(df)
    weekly_ranges = resample(pl.DataFrame(df), "7d", pd.Timedelta(days=6))
    weekly_ranges = weekly_ranges.to_pandas()

    breakouts = await find_breakouts(df, weekly_ranges)
    breakouts_pandas = pd.DataFrame(breakouts)
    # breakouts_polars = pl.DataFrame(breakouts)
    return breakouts_pandas


tradebook = await trade()
# tradebook

Current Week :  2022-01-12
Current Week :  2022-01-19
Current Week :  2022-01-26
Current Week :  2022-02-02
Current Week :  2022-02-09
Current Week :  2022-02-16
Current Week :  2022-02-23
Current Week :  2022-03-02
Current Week :  2022-03-09
Current Week :  2022-03-16
Current Week :  2022-03-23
Current Week :  2022-03-30
Current Week :  2022-04-06
Current Week :  2022-04-13
Current Week :  2022-04-20
Current Week :  2022-04-27
Current Week :  2022-05-04
Current Week :  2022-05-11
Current Week :  2022-05-18
Current Week :  2022-05-25
Current Week :  2022-06-01
Current Week :  2022-06-08
Current Week :  2022-06-15
Current Week :  2022-06-22
Current Week :  2022-06-29
Current Week :  2022-07-06
Current Week :  2022-07-13
Current Week :  2022-07-20
Current Week :  2022-07-27
Current Week :  2022-08-03
Current Week :  2022-08-10
Current Week :  2022-08-17
Current Week :  2022-08-24
Current Week :  2022-08-31
Current Week :  2022-09-07
Current Week :  2022-09-14
Current Week :  2022-09-21
C

In [34]:
tradebook

,Week,Week High,Week Low,Weekly Range,Current Week Open,Multiplier to Range,Entry Level Long,Entry Level Short,Break Type,Strike,Expiry,DTE,Contract,Option Type,Entry Time,Entry Price,Exit Time,Exit Price,Remark,Points,Slippage,Points w cs,Qty,PnL,PnL w cs,ROI%,ROI% w cs
0,2022-01-05,18562.6000,17981.5000,581.1000,18660.2500,40%,18892.6900,18427.8100,low,18900,2022-01-18,4,FINNIFTY2211818900CE,CE,2022-01-14 09:15:00,50.0500,2022-01-14 09:45:00,40.9500,SL Hit,9.1000,0.9100,8.1900,423.2804,3851.8519,3466.6667,0.3852,0.3467
1,2022-01-12,18747.3500,18412.7500,334.6000,18488.0500,40%,18621.8900,18354.2100,low,18600,2022-01-25,6,FINNIFTY22JAN18600CE,CE,2022-01-19 10:25:00,NaN,2022-01-25 15:30:00,NaN,Expiry Closing,NaN,NaN,NaN,430.1075,NaN,NaN,NaN,NaN
2,2022-01-19,18488.0500,17325.7000,1162.3500,17509.3000,40%,17974.2400,17044.3600,high,17050,2022-02-01,4,FINNIFTY2220117050PE,PE,2022-01-28 09:58:00,NaN,2022-02-01 15:30:00,NaN,Expiry Closing,NaN,NaN,NaN,469.2082,NaN,NaN,NaN,NaN
3,2022-01-26,18217.3500,17387.1000,830.2500,18207.1000,40%,18539.2000,17875.0000,low,18550,2022-02-08,1,FINNIFTY2220818550CE,CE,2022-02-07 10:58:00,NaN,2022-02-08 15:30:00,NaN,Expiry Closing,NaN,NaN,NaN,431.2668,NaN,NaN,NaN,NaN
4,2022-02-02,18427.3500,17379.9000,1047.4500,17761.5500,40%,18180.5300,17342.5700,low,18200,2022-02-15,1,FINNIFTY2221518200CE,CE,2022-02-14 09:16:00,0.1000,2022-02-15 15:30:00,0.0500,Expiry Closing,0.0500,0.0015,0.0485,439.5604,21.9780,21.3187,0.0022,0.0021
5,2022-02-09,18145.4500,16976.8500,1168.6000,17654.8500,40%,18122.2900,17187.4100,low,18100,2022-02-22,0,FINNIFTY22FEB18100CE,CE,2022-02-22 09:15:00,NaN,2022-02-22 15:30:00,NaN,Expiry Closing,NaN,NaN,NaN,441.9890,NaN,NaN,NaN,NaN
6,2022-02-16,17772.0500,17136.0000,636.0500,17586.3500,40%,17840.7700,17331.9300,low,17850,2022-02-28,4,FINNIFTY22FEB17850CE,CE,2022-02-24 09:15:00,NaN,2022-02-28 15:30:00,NaN,Expiry Closing,NaN,NaN,NaN,448.1793,NaN,NaN,NaN,NaN
7,2022-02-23,17628.0000,16451.7000,1176.3000,16573.6500,40%,17044.1700,16103.1300,low,17050,2022-03-08,4,FINNIFTY2230817050CE,CE,2022-03-04 09:16:00,NaN,2022-03-08 15:30:00,NaN,Expiry Closing,NaN,NaN,NaN,469.2082,NaN,NaN,NaN,NaN
8,2022-03-02,16727.8500,15043.9000,1683.9500,15565.4000,40%,16238.9800,14891.8200,high,14900,2022-03-15,5,FINNIFTY2231514900PE,PE,2022-03-10 09:15:00,37.5000,2022-03-15 15:30:00,1.0000,Expiry Closing,36.5000,0.3850,36.1150,536.9128,19597.3154,19390.6040,1.9597,1.9391
9,2022-03-09,16651.6000,15429.9500,1221.6500,16596.5000,40%,17085.1600,16107.8400,high,16100,2022-03-22,5,FINNIFTY2232216100PE,PE,2022-03-17 10:08:00,0.5000,2022-03-22 15:30:00,0.0500,Expiry Closing,0.4500,0.0055,0.4445,496.8944,223.6025,220.8696,0.0224,0.0221


In [35]:
# positive = tradebook[tradebook['Points']<0]
# positive['Points'].sum()
tradebook["ROI% w cs"].sum()
# no_value = tradebook[(tradebook['ROI%'] > 0) | (tradebook['ROI%'] < 0)]
# len(no_value)

56.34310782647867

In [36]:
# x = tradebook['Entry Time'].iloc[0].year
tradebook["Entry Time"] = pd.to_datetime(tradebook["Entry Time"])
tradebook["Trade Year"] = tradebook["Entry Time"].dt.year

In [37]:
stats_df8 = pd.DataFrame(
    index=range(2017, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tradebook
# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["ROI% w cs"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["ROI% w cs"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["ROI% w cs"] > 0]["ROI% w cs"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["ROI% w cs"] < 0]["ROI% w cs"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["ROI% w cs"].cumsum() - year_trades["ROI% w cs"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["ROI% w cs"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% w cs"] > 0][
    "ROI% w cs"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% w cs"] < 0][
    "ROI% w cs"
].mean()
overall_max_drawdown = (
    combined_df_sorted["ROI% w cs"].cumsum()
    - combined_df_sorted["ROI% w cs"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,0.0000,0,NaN,NaN,NaN,NaN,NaN
2018,0.0000,0,NaN,NaN,NaN,NaN,NaN
2019,0.0000,0,NaN,NaN,NaN,NaN,NaN
2020,0.0000,0,NaN,NaN,NaN,NaN,NaN
2021,0.0000,0,NaN,NaN,NaN,NaN,NaN
2022,23.5283,47,46.8085,1.0892,-0.4350,-0.4350,54.0858
2023,20.9252,47,87.2340,0.8278,-2.1692,-6.3634,3.2884
2024,11.8896,13,61.5385,1.6509,-0.6590,-1.0344,11.4940
Overall,56.3431,107.0000,66.3551,1.0016,-1.6409,-6.3634,8.8543


50% MOVE VARIATION FROM CURRENT OPEN, ENTRY AT OPP LEVELS, SL AT PRV. WK H/L

In [38]:
# stats_of_trades = tradebook
# stats_of_trades['Cumulative ROI%'] = stats_of_trades.groupby('Option Type')['ROI%'].cumsum()
# stats_of_trades['Running Max ROI%'] = stats_of_trades.groupby('Option Type')['Cumulative ROI%'].cummax()
# stats_of_trades['Drawdown'] = stats_of_trades['Cumulative ROI%'] - stats_of_trades['Running Max ROI%']
# max_dd_distribution = stats_of_trades.groupby('Option Type')['Drawdown'].min().reset_index()
# max_dd_distribution.rename(columns={'Drawdown': 'Max Drawdown'}, inplace=True)
# roi_distribution = stats_of_trades.groupby('Option Type')['ROI%'].sum().reset_index()
# grouped = pd.merge(roi_distribution, max_dd_distribution, on='Option Type')
# grouped['ROI/DD Ratio'] = grouped['ROI%'] / grouped['Max Drawdown'].abs()
# grouped

In [39]:
# tradebook.to_csv('NIFTY Final TB JJMS RBOS 40pct.csv')
# tradebook2 = pd.read_csv("NIFTY Final TB JJMS RBOS 40pct.csv")
tradebook2 = tradebook
tradebook2["Expiry"] = pd.to_datetime(tradebook2["Expiry"])
tradebook2["Entry Time"] = pd.to_datetime(tradebook2["Entry Time"])
tradebook2["Exit Time"] = pd.to_datetime(tradebook2["Exit Time"])

In [40]:
def round_to_nearest_50(value):
    return 50 * round(value / 50)


async def add_hedges(df):
    # df.drop(columns=['ROI%', 'Trade Year'], inplace=True)
    hedge_pct = 2
    index_name = "FINNIFTY"
    index_str_for_opt = "finnifty"

    # df['Hedge Strike'] = df.apply(lambda row: row['Strike'] + round_to_nearest_100((hedge_pct / 100) * row['Strike']) if 'C' in row['Contract'] else row['Strike'] - round_to_nearest_100((hedge_pct / 100) * row['Strike']), axis=1)
    df["Hedge Strike"] = df.apply(
        lambda row: (
            round_to_nearest_50(row["Strike"] * (1 + hedge_pct / 100))
            if row["Option Type"] == "CE"
            else round_to_nearest_50(row["Strike"] * (1 - hedge_pct / 100))
        ),
        axis=1,
    )

    df["Hedge Contract"] = df.apply(
        lambda row: get_option_contract_name2(
            index_name, row["Hedge Strike"], row["Expiry"], row["Option Type"]
        ),
        axis=1,
    )

    df["Hedge Entry Price"] = np.nan
    df["Hedge Exit Price"] = np.nan

    for i in range(0, len(df)):
        print(df.iloc[i]["Week"])
        hedge_strike = int(df.iloc[i]["Hedge Strike"])
        hedge_expiry = df.iloc[i]["Expiry"]
        hedge_opt_type = df.iloc[i]["Option Type"][-2:-1]
        hedge_entry_datetime = df.iloc[i]["Entry Time"]
        hedge_exit_datetime = df.iloc[i]["Exit Time"]

        hedge_df = await fetch_data(
            index=index_str_for_opt,
            start_date=hedge_entry_datetime.date(),
            end_date=hedge_exit_datetime.date(),
            start_time=hedge_entry_datetime.time(),
            end_time=hedge_exit_datetime.time(),
            expiry=hedge_expiry,
            strike=hedge_strike,
            asset_class=hedge_opt_type,
        )
        if not isinstance(hedge_df, str) and hedge_df is not None:
            hedge_df = hedge_df.to_pandas()
        else:
            print(hedge_df)
            df.loc[i, "Hedge Entry Price"] = 0
            df.loc[i, "Hedge Exit Price"] = 0
            continue
        # print(hedge_df)

        hedge_entry_price = hedge_df.iloc[0]["c"]
        hedge_exit_price = hedge_df.iloc[-1]["c"]

        print(hedge_entry_price, hedge_exit_price)

        df.loc[i, "Hedge Entry Price"] = hedge_entry_price
        df.loc[i, "Hedge Exit Price"] = hedge_exit_price

    return df


tb_with_hedge_tmp = await add_hedges(tradebook2)
# tb_with_hedge

2022-01-05
None
2022-01-12
None
2022-01-19
12.0 0.05
2022-01-26
0.5 0.1
2022-02-02
None
2022-02-09
None
2022-02-16
None
2022-02-23
3.5 0.1
2022-03-02
4.95 0.05
2022-03-09
5.35 0.05
2022-03-16
None
2022-03-23
1.75 0.05
2022-04-06
12.05 0.05
2022-04-20
1.7 4.0
2022-04-20
None
2022-04-27
None
2022-05-04
9.0 0.05
2022-05-11
9.0 0.05
2022-05-18
None
2022-05-25
0.1 0.05
2022-06-01
2.0 0.05
2022-06-15
None
2022-06-22
None
2022-06-29
6.0 0.05
2022-07-06
None
2022-07-13
None
2022-07-20
None
2022-08-03
9.5 0.05
2022-08-10
1.15 0.05
2022-08-17
None
2022-08-17
None
2022-08-24
None
2022-08-31
10.15 0.05
2022-09-07
None
2022-09-14
None
2022-09-28
1.0 0.05
2022-10-05
None
2022-10-12
1.0 0.05
2022-10-19
0.4 0.05
2022-10-26
5.0 0.5
2022-11-02
3.1 0.05
2022-11-02
3.9 2.25
2022-11-16
2.8 0.05
2022-11-23
10.25 0.05
2022-11-30
0.4 0.05
2022-12-07
5.45 0.05
2022-12-14
9.5 0.05
2022-12-28
7.05 0.05
2023-01-11
8.95 0.05
2023-01-18
5.4 0.05
2023-01-25
14.2 0.05
2023-02-08
10.5 24.65
2023-02-08
2.25 0.1
2023-02

In [41]:
# tb_with_hedge = pd.read_csv('JJMS TB w Hedges 1%.csv')
# tb_with_hedge

In [42]:
tb_with_hedge = tb_with_hedge_tmp

In [43]:
tb_with_hedge["Qty"] = tb_with_hedge["Qty"] * 3
tb_with_hedge["Hedge Points"] = (
    tb_with_hedge["Hedge Exit Price"] - tb_with_hedge["Hedge Entry Price"]
)
tb_with_hedge["PnL"] = tb_with_hedge["Points"] * tb_with_hedge["Qty"]
tb_with_hedge["PnL w cs"] = tb_with_hedge["Points w cs"] * tb_with_hedge["Qty"]
tb_with_hedge["Hedge PnL"] = tb_with_hedge["Hedge Points"] * tb_with_hedge["Qty"]
tb_with_hedge["Total PnL"] = tb_with_hedge["PnL w cs"] + tb_with_hedge["Hedge PnL"]

In [44]:
tb_with_hedge["Total ROI%"] = tb_with_hedge["Total PnL"] * 100 / 1000000
tb_with_hedge["Entry Time"] = pd.to_datetime(tb_with_hedge["Entry Time"])
tb_with_hedge["Trade Year"] = tb_with_hedge["Entry Time"].dt.year

In [45]:
# stats_of_trades = tb_with_hedge
# stats_of_trades["Cumulative ROI%"] = stats_of_trades.groupby("Option Type")[
#     "Total ROI%"
# ].cumsum()
# stats_of_trades["Running Max ROI%"] = stats_of_trades.groupby("Option Type")[
#     "Cumulative ROI%"
# ].cummax()
# stats_of_trades["Drawdown"] = (
#     stats_of_trades["Cumulative ROI%"] - stats_of_trades["Running Max ROI%"]
# )
# max_dd_distribution = (
#     stats_of_trades.groupby("Option Type")["Drawdown"].min().reset_index()
# )
# max_dd_distribution.rename(columns={"Drawdown": "Max Drawdown"}, inplace=True)
# roi_distribution = (
#     stats_of_trades.groupby("Option Type")["Total ROI%"].sum().reset_index()
# )
# grouped = pd.merge(roi_distribution, max_dd_distribution, on="Option Type")
# grouped["ROI/DD Ratio"] = grouped["Total ROI%"] / grouped["Max Drawdown"].abs()
# grouped

In [46]:
stats_df8 = pd.DataFrame(
    index=range(2022, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tb_with_hedge
# Iterate over each year
for year in range(2022, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["Total ROI%"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["Total ROI%"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["Total ROI%"] > 0]["Total ROI%"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["Total ROI%"] < 0]["Total ROI%"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["Total ROI%"].cumsum() - year_trades["Total ROI%"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["Total ROI%"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["Total ROI%"] > 0][
    "Total ROI%"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["Total ROI%"] < 0][
    "Total ROI%"
].mean()
overall_max_drawdown = (
    combined_df_sorted["Total ROI%"].cumsum()
    - combined_df_sorted["Total ROI%"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2022,61.9167,47,44.6809,3.0550,-1.1189,-1.5140,40.8954
2023,41.7445,47,87.2340,1.9068,-6.0721,-17.2545,2.4193
2024,24.0320,13,61.5385,3.3911,-1.5483,-1.9525,12.3084
Overall,127.6933,107.0000,65.4206,2.4209,-4.1767,-17.2545,7.4006


In [47]:
tb_with_hedge.to_csv('FINNIFTY TB RBOS w hedge 2pct_3x.csv')

In [56]:
a = tb_with_hedge["PnL"].sum()
b = tb_with_hedge["Hedge PnL"].sum()
c = tb_with_hedge["Total PnL"].sum()
d = tb_with_hedge["Total ROI%"].sum()
print(a, b, c, d)

2123508.9437905895 -634730.0671854533 1540879.6684031477 154.08796684031478


In [26]:
# tb_with_hedge.to_csv('JJMS TB w Hedges 1%.csv')

In [27]:
# # tb_with_hedge['DD%'] = (tb_with_hedge['Total ROI%'].cumsum() - tb_with_hedge['Total ROI%'].cumsum().cummax())
# tb_with_hedge.drop(columns=['Cumulative ROI%', 'Running Max ROI%', 'Drawdown'], inplace=True)
# tb_with_hedge

In [28]:
# tb_with_hedge["ROI%"] = tb_with_hedge["PnL"] * 100 / 1000000

In [29]:
# tb_with_hedge["Hedge PnL"].sum()

In [66]:
stats_df9 = pd.DataFrame(
    index=range(2017, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tb_with_hedge
# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["Total ROI%"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["Total ROI%"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["Total ROI%"] > 0]["Total ROI%"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["Total ROI%"] < 0]["Total ROI%"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["Total ROI%"].cumsum() - year_trades["Total ROI%"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df9.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df9["Total ROI"].sum()
overall_total_trades = stats_df9["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["ROI% w cs"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["Total ROI%"] > 0][
    "Total ROI%"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["Total ROI%"] < 0][
    "Total ROI%"
].mean()
overall_max_drawdown = (
    combined_df_sorted["Total ROI%"].cumsum()
    - combined_df_sorted["Total ROI%"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df9.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df9

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,7.9201,54,18.5185,1.6010,-2.0226,-2.8246,2.8040
2018,14.8819,54,20.3704,1.7223,-4.0635,-4.0635,3.6624
2019,50.5409,48,70.8333,2.0289,-4.6102,-8.2073,6.1581
2020,49.8057,51,82.3529,2.5199,-7.0038,-20.7698,2.3980
2021,35.5399,51,86.2745,1.6941,-5.5716,-19.4401,1.8282
2022,55.4865,57,82.4561,1.5963,-1.9538,-7.6280,7.2740
2023,42.4170,50,90.0000,1.0842,-2.1240,-4.0808,10.3943
2024,4.0178,14,78.5714,1.5830,-4.4650,-13.1456,0.3056
Overall,260.6097,379.0000,66.7546,1.7440,-4.1233,-20.7698,12.5475


In [31]:
# tradebook["Cumulative ROI%"] = tradebook["ROI% w cs"].cumsum()
# tradebook["Running Max ROI%"] = tradebook["Cumulative ROI%"].cummax()
# tradebook["Drawdown"] = tradebook["Cumulative ROI%"] - tradebook["Running Max ROI%"]
# tradebook

In [32]:
# tradebook.to_csv("2020 Trades JJMS OS.csv")